### Initilize Tree:

initilize tree size to 4 times the array length so that the resulting binary tree is balanced.

In [14]:
arr = [18, 17, 13, 19, 15, 11, 20, 12]
t = [None] * 4 * len(arr)     

### Build Segment Tree for range min query

In [15]:
def left(node):
    return 2*node + 1

def right(node):
    return 2*node + 2

def build(arr, node, start, end):
    if start == end:
        t[node] = arr[start]
    else:
        mid = (start + end) // 2
        build(arr, left(node), start, mid)     
        build(arr, right(node), mid + 1, end) 
        t[node] = min(t[left(node)], t[right(node)])


In [16]:
build(arr, 0, 0, 7)
print(t)

[11, 13, 11, 17, 13, 11, 12, 18, 17, 13, 19, 15, 11, 20, 12, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


### Range min query

In [41]:
def range_minimum_query(node, segx, segy, qx, qy):
    '''
    returns the minimum number in range(qx,qy)
    segx and segy represent the segment index

    '''
    if qx > segy or qy < segx:      # query COMPLETELY out of range
        return float("inf")
    elif qx <= segx <= segy <= qy:  # segment range inside of query range
        return t[node]
    else:
        return min(
            range_minimum_query(left(node), segx, (segx + segy) // 2, qx, qy),
            range_minimum_query(right(node), (segx + segy) // 2 + 1, segy, qx, qy)
        )

In [42]:
range_minimum_query(0, 0, 7, 1, 4)

11

### Update Segment tree

update all the elements in range [qx, qy] to val.

In [73]:
def update(node, segx, segy, qx, qy, val):
    if qx > segy or qy < segx:      # query COMPLETELY out of range
        return     
    elif segx == segy:
        t[node] = val
    else:
        update(left(node), segx, (segx + segy) // 2, qx, qy, val),
        update(right(node), (segx + segy) // 2 + 1, segy, qx, qy, val)
 
        t[node] = min(t[left(node)], t[right(node)])

In [74]:
print('oldtree: ', t)
update(0, 0, 7, 1, 2, 6)
print('newtree: ', t)

oldtree:  [11, 13, 11, 17, 13, 11, 12, 18, 17, 13, 19, 15, 11, 20, 12, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
newtree:  [6, 6, 11, 6, 6, 11, 12, 18, 6, 6, 19, 15, 11, 20, 12, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


### Lazy propagation

When the problem is dynamic and involves many updates to the tree, we can use lazy propagation to decrease the number of times a node is visited.

Lazy propagation involves maintaining a seprate array of the same size as our tree array. Upon visiting a parent node, we update the value of that node only, and store updates for the children in the lazy array. 

Only when those children are visited in another operation do we consult if the node is lazy (not up to date, the value in it needs correction) and if so correct its value from the lazy array, and lest we forget, propagate the lazy values to its children.

In [7]:
lazy = [0]*4*len(arr)

In [11]:
def lazy_update(node, segx, segy, qx, qy, val):
    
    #make sure all propagation was done at node, if not
    #then update node and mark its children for lazy propagation
    
    #This needs to be done BEFORE checking for query out of range or not,
    #because we will need the updated value when returning for the parent anyway !!!!
    if lazy[node]:
        t[node] = lazy[node]
        if segx != segy:
            lazy[left(node)] = lazy[node]
            lazy[right(node)] = lazy[node]
        lazy[node] = 0
        
    # query COMPLETELY out of range
    if segx > segy or qx > segy or qy < segx:      
        return
    
    # query completely within range 
    if qx <= segx <= segy <= qy:
        t[node] = val
        if segx != segy:
            lazy[left(node)] = val
            lazy[right(node)] = val
        
    else:
        lazy_update(left(node), segx, (segx + segy) // 2, qx, qy, val),
        lazy_update(right(node), (segx + segy) // 2 + 1, segy, qx, qy, val)
 
        t[node] = min(t[left(node)], t[right(node)])

We need to modify the query function to work with lazy propagation by adding the logic that updates our node if it was lazy:

In [12]:
def lazy_range_minimum_query(node, segx, segy, qx, qy):
    '''
    returns the minimum number in range(qx,qy)
    segx and segy represent the segment index

    '''
    
    
    if lazy[node]:
        t[node] = lazy[node]
        if segx != segy:
            lazy[left(node)] = lazy[node]
            lazy[right(node)] = lazy[node]
        lazy[node] = 0

    
    if qx > segy or qy < segx:      # query COMPLETELY out of range
        return float("inf")
    elif qx <= segx <= segy <= qy:  # segment range inside of query range
        return t[node]
    else:
        return min(
            lazy_range_minimum_query(left(node), segx, (segx + segy) // 2, qx, qy),
            lazy_range_minimum_query(right(node), (segx + segy) // 2 + 1, segy, qx, qy)
        )


In [21]:
lazy_update(0, 0, 7, 2, 3, 1)

In [22]:
lazy_range_minimum_query(0, 0, 7, 0, 3)

1

### Resources:

https://www.youtube.com/watch?v=xuoQdt5pHj0

https://visualgo.net/en/segmenttree?slide=1

https://leetcode.com/articles/a-recursive-approach-to-segment-trees-range-sum-queries-lazy-propagation/